In [58]:
#use classicalvenv
import dimod
import json
import numpy as np

In [59]:
def load_ising(file_path):
    with open(file_path, "r") as f:
        all_isings_data = json.load(f)  
    return all_isings_data

def save_solutions_to_json(solutions_dict, output_filepath):
    output_list = []
    for instance_id, solutions_list in solutions_dict.items():            
        solutions = [sol[0] for sol in solutions_list]
        
        # The cost is the same for all solutions of an instance
        cost = solutions_list[0][1]

        formatted_entry = {
            "instance_id": instance_id,
            "solutions": solutions, # Stored as a list of strings
            "cost": float(cost)
        }
        
        output_list.append(formatted_entry)
        
    with open(output_filepath, 'w') as f:
        json.dump(output_list, f, indent=2)

def convert_to_dimod_ising(all_isings_data):
    dimod_ising = {}
    for instance in all_isings_data:
        instance_id = instance["instance_id"]
        terms = instance["terms"]
        weights = instance["weights"]

        linear_bias = {}
        quadratic_bias = {}

        for term, weight in zip(terms, weights):
            if len(term) == 1:
                variable = f'x{term[0]}'
                linear_bias[variable] = weight
            elif len(term) == 2:
                var1 = f'x{term[0]}'
                var2 = f'x{term[1]}'
                quadratic_bias[(var1, var2)] = weight

        dimod_ising[instance_id] = (linear_bias, quadratic_bias)
    return dimod_ising

def extract_lowest_energy_solutions(allSolutions):
    results = []
    lowest_energy = allSolutions.first.energy
    for datum in allSolutions.data():
        if datum.energy > lowest_energy:
            break
        sorted_keys = sorted(datum.sample.keys(), key=lambda v: int(v[1:]))
        variable_values = []
        for key in sorted_keys:
            val = datum.sample[key]
            if val > 0:
                variable_values.append(f"+{val}")
            else:
                variable_values.append(str(val))
        
        variable_string = ",".join(variable_values)
        energy = float(datum.energy)
        results.append((variable_string, energy))
            
    return results

In [60]:
#////////// variables //////////
isingBatchFile = "batch_Ising_data_Knapsack_6_items_9q_.json"
#isingBatchFile = "batch_Ising_data_TSP_9q_.json"
inputFolder = "isingBatches/"

In [61]:
isings = load_ising(inputFolder+isingBatchFile)
dimodIsings = convert_to_dimod_ising(isings)
print(dimodIsings)

{1: ({'x0': -60.0, 'x1': -120.0, 'x2': -240.0, 'x3': -59.0, 'x4': -58.5, 'x5': -238.5, 'x6': -298.0, 'x7': -177.5, 'x8': -299.0}, {('x0', 'x1'): 10.0, ('x0', 'x2'): 20.0, ('x1', 'x2'): 40.0, ('x3', 'x4'): 5.0, ('x3', 'x5'): 20.0, ('x3', 'x6'): 25.0, ('x3', 'x7'): 15.0, ('x8', 'x3'): 25.0, ('x4', 'x5'): 20.0, ('x4', 'x6'): 25.0, ('x4', 'x7'): 15.0, ('x8', 'x4'): 25.0, ('x5', 'x6'): 100.0, ('x5', 'x7'): 60.0, ('x8', 'x5'): 100.0, ('x6', 'x7'): 75.0, ('x8', 'x6'): 125.0, ('x8', 'x7'): 75.0, ('x0', 'x3'): 5.0, ('x0', 'x4'): 5.0, ('x0', 'x5'): 20.0, ('x0', 'x6'): 25.0, ('x0', 'x7'): 15.0, ('x0', 'x8'): 25.0, ('x1', 'x3'): 10.0, ('x1', 'x4'): 10.0, ('x1', 'x5'): 40.0, ('x1', 'x6'): 50.0, ('x1', 'x7'): 30.0, ('x8', 'x1'): 50.0, ('x2', 'x3'): 20.0, ('x2', 'x4'): 20.0, ('x2', 'x5'): 80.0, ('x2', 'x6'): 100.0, ('x2', 'x7'): 60.0, ('x8', 'x2'): 100.0}), 2: ({'x0': -60.0, 'x1': -120.0, 'x2': -240.0, 'x3': -179.0, 'x4': -238.0, 'x5': -59.0, 'x6': -117.5, 'x7': -238.0, 'x8': -178.5}, {('x0', 'x1'): 

In [62]:
solver = dimod.ExactSolver()

globalOptima = {}
for instance, biases in dimodIsings.items():
    #print(instance)
    I = dimodIsings[instance]
    sampleset = solver.sample_ising(I[0], I[1])
    instanceGlobalOptima = extract_lowest_energy_solutions(sampleset)
    globalOptima[instance] = instanceGlobalOptima

print(globalOptima)

{1: [('+1,-1,+1,-1,-1,+1,+1,-1,+1', -605.5)], 2: [('+1,+1,+1,+1,+1,-1,-1,+1,-1', -550.0)], 3: [('+1,+1,+1,+1,-1,-1,+1,+1,-1', -916.0)], 4: [('+1,+1,+1,-1,+1,-1,+1,-1,+1', -422.0), ('-1,+1,+1,+1,+1,-1,+1,-1,-1', -422.0), ('-1,+1,+1,+1,+1,-1,-1,-1,+1', -422.0), ('+1,+1,+1,-1,+1,+1,+1,-1,-1', -422.0), ('+1,+1,+1,-1,+1,+1,-1,-1,+1', -422.0), ('-1,+1,+1,+1,+1,+1,-1,-1,-1', -422.0)], 5: [('+1,+1,+1,-1,-1,+1,+1,+1,-1', -730.5)], 6: [('+1,+1,+1,-1,+1,+1,-1,+1,-1', -573.0)], 7: [('+1,+1,+1,-1,+1,-1,+1,+1,-1', -413.0), ('-1,+1,+1,+1,-1,-1,+1,+1,+1', -413.0)], 8: [('+1,-1,+1,+1,-1,+1,-1,-1,-1', -301.5)], 9: [('+1,+1,+1,-1,-1,+1,+1,+1,-1', -856.0)], 10: [('+1,+1,+1,+1,-1,-1,-1,-1,-1', -92.5)], 11: [('-1,+1,+1,-1,+1,+1,-1,-1,-1', -245.0)], 12: [('-1,+1,+1,-1,-1,+1,-1,+1,+1', -590.5)], 13: [('+1,+1,+1,+1,+1,-1,-1,-1,+1', -369.0)], 14: [('+1,+1,+1,-1,+1,+1,+1,-1,-1', -572.0)], 15: [('+1,+1,+1,-1,+1,+1,+1,-1,-1', -248.0)], 16: [('+1,+1,+1,+1,+1,-1,+1,-1,-1', -393.5), ('+1,-1,+1,+1,-1,-1,+1,+1,-1', -39

In [63]:
output_file = inputFolder+'solved_'+isingBatchFile
save_solutions_to_json(globalOptima, output_file)

In [64]:
print(sampleset.slice(10))
print(sampleset)

  x0 x1 x2 x3 x4 x5 x6 x7 x8 energy num_oc.
0 +1 +1 +1 +1 -1 -1 +1 +1 -1 -648.0       1
1 -1 +1 +1 +1 -1 +1 -1 +1 -1 -647.0       1
2 +1 +1 +1 +1 -1 +1 +1 -1 -1 -644.0       1
3 +1 +1 -1 +1 -1 +1 +1 +1 -1 -643.0       1
4 -1 +1 +1 +1 -1 -1 +1 +1 +1 -643.0       1
5 -1 +1 +1 +1 +1 -1 +1 +1 -1 -643.0       1
6 +1 -1 +1 +1 +1 +1 -1 +1 -1 -642.0       1
7 +1 +1 +1 -1 +1 +1 +1 +1 -1 -642.0       1
8 +1 +1 +1 -1 -1 +1 +1 +1 +1 -642.0       1
9 +1 -1 +1 +1 -1 +1 -1 +1 +1 -642.0       1
['SPIN', 10 rows, 10 samples, 9 variables]
    x0 x1 x2 x3 x4 x5 x6 x7 x8 energy num_oc.
138 +1 +1 +1 +1 -1 -1 +1 +1 -1 -648.0       1
203 -1 +1 +1 +1 -1 +1 -1 +1 -1 -647.0       1
74  +1 +1 +1 +1 -1 +1 +1 -1 -1 -644.0       1
148 -1 +1 +1 +1 +1 -1 +1 +1 -1 -643.0       1
178 +1 +1 -1 +1 -1 +1 +1 +1 -1 -643.0       1
372 -1 +1 +1 +1 -1 -1 +1 +1 +1 -643.0       1
165 +1 +1 +1 -1 +1 +1 +1 +1 -1 -642.0       1
214 +1 -1 +1 +1 +1 +1 -1 +1 -1 -642.0       1
310 +1 -1 +1 +1 -1 +1 -1 +1 +1 -642.0       1
325 +1 +1 +1 